# Trasnfer of Learning Assignment
This assignment you should complete this notebook. This is the transfer of learning assignment. You should transfer learning from ResNet50 and predict from MNIST dataset. Codes for loading and assigning train and test sets are already provided. You need to use your university ID for random seed. Also, classification should be developed with decision tress

<div style="display: inline-block">
  <img src="https://thedatafrog.com/static/blog/images/2019/10/dessin_transfer_learning_crop-1-1024x794.jpg" width="300" height="200">
</div>
<div style="display: inline-block">
  <img src="https://miro.medium.com/v2/resize:fit:786/format:webp/0*tH9evuOFqk8F41FG.png" width="300" height="200">
</div>

Image Source: thedatafrog.com & towardsdatascience.com

part 1: Install the required packages!!

The following function is a command used to restart the Python interpreter running on the Spark driver node. The purpose of this command is to reload any libraries or modules that have been updated since the interpreter was last started, so that the updated code can be used in subsequent PySpark jobs.

In [0]:
dbutils.library.restartPython()

It's time to import packages and starting a spark session

In [0]:
# part 2: you need to figure out which libraries you should import


# Create Spark session
spark = SparkSession.builder.appName("assignment").getOrCreate()

we use the subsets of the dataset to use a smaller portion of the images for training and testing. This is only for making it excutable by low power PCs and/or community version of Databricks!

In [0]:
# Load mnist dataset from TensorFlow
(train_images, train_labels), (test_images, test_labels)  = datasets.mnist.load_data()
# Subset the dataset
# part 3: use your numerical part of your ID as the random seed and then randomly pick 1000 observations of train set and 100 observations of the test set
# then instead of training on whole dataset just do training and evaluation on the subsets
random.seed(....) 



This creates Spark DataFrames from the NumPy arrays by flattening the image arrays, converting them to Vectors, and creating tuples with the corresponding labels.

In [0]:
# Create Spark DataFrames from NumPy arrays
train_df = spark.createDataFrame([(Vectors.dense(img.flatten().tolist()), int(label)) for img, label in zip(train_images, train_labels)], ["features", "label"])
test_df = spark.createDataFrame([(Vectors.dense(img.flatten().tolist()), int(label)) for img, label in zip(test_images, test_labels)], ["features", "label"])

In [0]:
# Extract features from images using pre-trained model
model = .... # part 4: complete this part with ResNet50 model

def extract_features(img):
    # Resize image to (224, 224, 3)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    # Preprocess image for ResNet50 model
    img = preprocess_input(img)
    # Extract features from image using ResNet50 model
    features = model.predict(img.reshape(1, 224, 224, 3)).ravel()
    # Return features as DenseVector
    return DenseVector(features)

# Convert NumPy arrays to list of images
train_images_list = [train_images[i] for i in range(train_images.shape[0])]
test_images_list = [test_images[i] for i in range(test_images.shape[0])]

# Extract features from images and convert to Spark DataFrames
train_df = spark.createDataFrame([(extract_features(img), int(label)) for img, label in zip(train_images_list, train_labels)], ["features", "label"])
test_df = spark.createDataFrame([(extract_features(img), int(label)) for img, label in zip(test_images_list, test_labels)], ["features", "label"])


In [0]:
# part 5: Train decision trees classification model on the extracted features
# then you need to report accuracy of train and test sets